<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/my_automatic_inspection_issues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requirements

## Dependencies

### Install dependencies

- datasets from Hugging Face
- transformers Hugging Face
- torch
- accelerate
- ntlk

In [1]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [2]:
!pip -q install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00


In [3]:
!pip -q install accelerate -U

In [4]:
!pip -q install nltk

### Import dependencies


- torch
- pandas
- numpy
- transformers
- sklearn
- datasets
- json
- string
- nltk

In [5]:
import torch
import pandas as pd
import numpy as np

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from datasets import load_dataset
import json

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from urllib.parse import urlparse

### NLTK Dependencies

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### Support Functions

In [8]:
def truncate_string(text, max_length=100, add_ellipsis=True):
  if len(text) <= max_length:
    return text

  truncated_text = text[:max_length]

  if add_ellipsis:
    truncated_text += "..."

  return truncated_text

def to_lowercase(text):
  return text.lower()

def remove_hyperlinks(text):
  tokens = nltk.word_tokenize(text)
  filtered_tokens = [token for token in tokens if not urlparse(token).scheme]
  return ' '.join(filtered_tokens)

def remove_punctuation(text):
  translator = str.maketrans('', '', string.punctuation)
  return text.translate(translator)

def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = text.split()
  filtered_words = [word for word in words if word not in stop_words]
  return ' '.join(filtered_words)

def preprocess_text(text):
  text = to_lowercase(text)
  text = remove_hyperlinks(text)
  #text = remove_punctuation(text)
  text = remove_stopwords(text)
  return text

In [9]:
def transform_dataset(coluna_uteis, minhas_colunas, df_x_issues_inspected):
	df_x_issues_inspected = df_x_issues_inspected[coluna_uteis]
	df_x_issues_inspected['label'] = df_x_issues_inspected['architectural_impact'].apply(lambda x: 1 if x == 'YES' else 0)
	df_x_issues_inspected['Textual_Type'] = 'AI_Yes'
	df_x_issues_inspected.loc[df_x_issues_inspected['label']==0, 'Textual_Type'] = 'AI_No'
	df_x_issues_inspected['SummaryDescriptionComments']= df_x_issues_inspected.apply(lambda row: row['summary'] + ' ' + row['description'] + ' ' + row['comments'],axis=1).values
	df_x_issues_inspected['processed_text'] = df_x_issues_inspected['SummaryDescriptionComments'].apply(preprocess_text)
	df_x_issues_inspected = df_x_issues_inspected[minhas_colunas]

	return df_x_issues_inspected

def perform_tests_in_model(X,y, label2id):
  list_my_metrics = list()
  for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {i+1}: Train Size {len(train_index)} | Test Size {len(test_index)}")
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    train_labels_encoded = [float(label2id[yi]) for yi in y_train]
    test_labels_encoded  = [float(label2id[yi]) for yi in y_test]

    X_train = [str(i) for i in X_train]
    X_test = [str(i) for i in X_test]

    unique_labels = set(label for label in y_train)
    label2id = {label: id for id, label in enumerate(unique_labels)}
    id2label = {id: label for label, id in label2id.items()}
    train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
    test_encodings  = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)

    train_labels_encoded = [float(label2id[yi]) for yi in y_train]
    test_labels_encoded  = [float(label2id[yi]) for yi in y_test]

    train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)

    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device_name)
    trainer = Trainer(
	      model=model,
	      args=training_args,
	      train_dataset=train_dataset,
	      eval_dataset=test_dataset,
	      compute_metrics=compute_metrics
	    )
    trainer.train()
    trainer.evaluate()

    predicted_results = trainer.predict(test_dataset)
    outputs = predicted_results.predictions.flatten().tolist()
    probas = [cap_number(x) for x in outputs]
    preds = np.array(np.array(probas) > 0.5, dtype=int)

    # roc_auc_score(test_labels_encoded, probas)
    folds[i] = {}
    folds[i]['pre'] = precision_score(test_labels_encoded, preds)
    folds[i]['rec'] = recall_score(test_labels_encoded, preds)
    folds[i]['acc'] = accuracy_score(test_labels_encoded, preds)
    folds[i]['auc'] = roc_auc_score(test_labels_encoded, probas)
    folds[i]['f1'] = f1_score(test_labels_encoded, preds)

    print(f"Fold {i+1}=> PRE: {folds[i]['pre']}; REC: {folds[i]['rec']}; ACC: {folds[i]['acc']}; F1S: {folds[i]['f1']}; AUC: {folds[i]['auc']}")
    item_metric = {'Fold':i+1, 'PRE':folds[i]['pre'], 'REC':folds[i]['rec'], 'ACC':folds[i]['acc'], 'F1S':folds[i]['f1'], 'AUC':folds[i]['auc']}
    list_my_metrics.append(item_metric)

  return list_my_metrics

In [10]:
coluna_uteis = ['issue_key', 'summary', 'description', 'comments', 'architectural_impact']

minhas_colunas = ['issue_key', 'SummaryDescriptionComments', 'processed_text', 'Textual_Type']

# Set up the environment for the AI ​​model

## Initial Setup

In [11]:
model_name = 'distilbert-base-uncased'
device_name = 'cuda'
#device_name = 'cpu'
max_length = 512
cached_model_directory_name = 'distilbert-ehbugs'

In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [13]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

## Support class to handle the dataset

In [14]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def cap_number(x):
    if x > 1:
      return 1
    elif x < 0:
      return 0
    else:
      return x

def compute_metrics(pred):
    labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    outputs = pred.predictions.flatten().tolist()
    probas = [cap_number(x) for x in outputs]
    preds = np.array(np.array(probas) > 0.5, dtype=int)
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

## Cleaning directories

Cleaning directories related to results and **logs**

In [15]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


In [17]:
!pip install --upgrade transformers

## Setup the arguments for training

In [18]:
# training_args = TrainingArguments(
#    num_train_epochs=3,              # total number of training epochs
#    per_device_train_batch_size=16,  # batch size per device during training
#    per_device_eval_batch_size=20,   # batch size for evaluation
#    learning_rate=5e-5,              # initial learning rate for Adam optimizer
#    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
#    weight_decay=0.01,               # strength of weight decay
#    output_dir='/content/results',   # output directory
#    logging_dir='/content/logs',     # directory for storing logs
#    logging_steps=150,               # number of steps to output logging (set lower because of small dataset size)
#    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
#)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [19]:
training_args = TrainingArguments(
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='/content/results',   # output directory
    logging_dir='/content/logs',     # directory for storing logs
    logging_steps=150,               # number of steps to output logging (set lower because of small dataset size)
    # The 'evaluation_strategy' parameter might not be available in your version of transformers
    # Remove it or update the library if needed
    # evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

In [20]:
unique_labels = {'AI_Yes', 'AI_No'}
label2id = {'AI_No': 0, 'AI_Yes': 1}
id2label = {0: 'AI_No', 1: 'AI_Yes'}

# A) Testes de Inspeção de Issues do Cassandra

## My Hugging Face Dataset

https://huggingface.co/datasets/armandoufpi/cassandraissuesgroundtruth


In [21]:
splits = {'train': 'train.jsonl', 'test': 'test.jsonl'}
df_treino = pd.read_json("hf://datasets/armandoufpi/cassandraissuesgroundtruth/" + splits["train"])
df_teste = pd.read_json("hf://datasets/armandoufpi/cassandraissuesgroundtruth/" + splits["test"])

In [24]:
dataset = pd.concat([df_treino, df_teste], axis=0)
df_cassandra_issues_inspected = dataset[['issue_key', 'summary', 'description', 'comments_text', 'architectural_impact']]
df_cassandra_issues_inspected.rename(columns={'comments_text': 'comments'}, inplace=True)

df_cassandra_issues_inspected = transform_dataset(coluna_uteis, minhas_colunas, df_cassandra_issues_inspected)
df_cassandra_issues_inspected.head()

<ipython-input-24-f079dc2c8aa2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected.rename(columns={'comments_text': 'comments'}, inplace=True)


,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,CASSANDRA-3489,EncryptionOptions should be instantiated As th...,"encryptionoptions instantiated title says , ot...",AI_No
1,CASSANDRA-16780,Log when writing many tombstones to a partitio...,log writing many tombstones partition log writ...,AI_No
2,CASSANDRA-5426,Redesign repair messages Many people have been...,redesign repair messages many people reporting...,AI_Yes
3,CASSANDRA-5121,system.peers.tokens is empty after node restar...,system.peers.tokens empty node restart using 2...,AI_No
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...,AI_Yes


## Create a StratifiedKFold

Stratified K-Fold cross-validator.

Provides train/test indices to split data in train/test sets.

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [25]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_cassandra_issues_inspected['processed_text'], df_cassandra_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

In [26]:
X

,processed_text
0,"encryptionoptions instantiated title says , ot..."
1,log writing many tombstones partition log writ...
2,redesign repair messages many people reporting...
3,system.peers.tokens empty node restart using 2...
4,sstablesinbounds might actually give sstables ...
...,...
21,duplicate rows returned clause repeated values...
22,examine shortening path length post-5202 cassa...
23,throw error auto_bootstrap : true bootstrappin...
24,support wrapped range queries want support sca...


In [27]:
y

,Textual_Type
0,AI_No
1,AI_No
2,AI_Yes
3,AI_No
4,AI_Yes
...,...
21,AI_No
22,AI_No
23,AI_No
24,AI_No


In [28]:
skf.split(X, y)

<generator object _BaseKFold.split at 0x7e13e7bbb0a0>

In [29]:
print(f"Running in {device_name}")

Running in cuda


## Train and Evaluate the Model

In [30]:
list_my_metrics_cassandra = perform_tests_in_model(X,y, label2id=label2id)

Fold 1: Train Size 180 | Test Size 46


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

Step,Training Loss


Fold 4=> PRE: 0.6666666666666666; REC: 1.0; ACC: 0.6666666666666666; F1S: 0.8; AUC: 0.7444444444444445
Fold 5: Train Size 181 | Test Size 45


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 5=> PRE: 0.6590909090909091; REC: 0.9666666666666667; ACC: 0.6444444444444445; F1S: 0.7837837837837838; AUC: 0.5711111111111111


## Cassandra issues results

In [31]:
df_my_metrics_cassandra_issues = pd.DataFrame(list_my_metrics_cassandra)
df_my_metrics_cassandra_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.673913,1.000000,0.673913,0.805195,0.744086
1,2,0.688889,1.000000,0.688889,0.815789,0.776498
2,3,0.666667,1.000000,0.666667,0.800000,0.653333
3,4,0.666667,1.000000,0.666667,0.800000,0.744444
4,5,0.659091,0.966667,0.644444,0.783784,0.571111


# B) Performing tests with Kafka issues

## Original dataset

In [32]:
!wget https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/kafka_issues_inspected.xlsx

--2025-04-29 14:35:26--  https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/kafka_issues_inspected.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147840 (144K) [application/octet-stream]
Saving to: ‘kafka_issues_inspected.xlsx’

kafka_issues_inspec 100%[===================>] 144.38K  --.-KB/s    in 0.02s   

2025-04-29 14:35:26 (6.97 MB/s) - ‘kafka_issues_inspected.xlsx’ saved [147840/147840]



In [33]:
df_kafka_issues_inspected = pd.read_excel('kafka_issues_inspected.xlsx')
df_kafka_issues_inspected = transform_dataset(coluna_uteis, minhas_colunas, df_kafka_issues_inspected)

<ipython-input-9-2b8ca721929e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_issues_inspected['label'] = df_x_issues_inspected['architectural_impact'].apply(lambda x: 1 if x == 'YES' else 0)
<ipython-input-9-2b8ca721929e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_issues_inspected['Textual_Type'] = 'AI_Yes'
<ipython-input-9-2b8ca721929e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [34]:
df_kafka_issues_inspected.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,KAFKA-1253,Implement compression in new producer \n It...,implement compression new producer seems incre...,AI_Yes
1,KAFKA-1250,Add slf4j logging to new producer Currently t...,add slf4j logging new producer currently loggi...,AI_No
2,KAFKA-1498,new producer performance and bug improvements ...,new producer performance bug improvements seen...,AI_No
3,KAFKA-2313,javadoc fix for KafkaConsumer deserialization ...,javadoc fix kafkaconsumer deserialization kafk...,AI_No
4,KAFKA-2123,Make new consumer offset commit API use callba...,make new consumer offset commit api use callba...,AI_Yes


## Cleaning directories

In [35]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Create a StratifiedKFold

In [36]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_kafka_issues_inspected['processed_text'], df_kafka_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

skf.split(X, y)

print(f"Running in {device_name}")

Running in cuda


## Performing tests in the model

In [37]:
list_my_metrics_kafka = perform_tests_in_model(X,y, label2id=label2id)

Fold 1: Train Size 143 | Test Size 36


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 1=> PRE: 0.5833333333333334; REC: 1.0; ACC: 0.5833333333333334; F1S: 0.7368421052631579; AUC: 0.5555555555555556
Fold 2: Train Size 143 | Test Size 36


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss


Fold 4=> PRE: 0.6111111111111112; REC: 1.0; ACC: 0.6111111111111112; F1S: 0.7586206896551724; AUC: 0.4772727272727273
Fold 5: Train Size 144 | Test Size 35


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 5=> PRE: 0.5882352941176471; REC: 0.9523809523809523; ACC: 0.5714285714285714; F1S: 0.7272727272727273; AUC: 0.5850340136054422


## Kafka issues results

In [38]:
df_my_metrics_kafka_issues = pd.DataFrame(list_my_metrics_kafka)
df_my_metrics_kafka_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.583333,1.000000,0.583333,0.736842,0.555556
1,2,0.583333,1.000000,0.583333,0.736842,0.736508
2,3,0.611111,1.000000,0.611111,0.758621,0.597403
3,4,0.611111,1.000000,0.611111,0.758621,0.477273
4,5,0.588235,0.952381,0.571429,0.727273,0.585034


# C) Testes de Inspeção de Issues do Hadoop

## Original Dataset

In [39]:
!wget https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/hadoop_issues_inspected.xlsx

--2025-04-29 14:40:58--  https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/hadoop_issues_inspected.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137652 (134K) [application/octet-stream]
Saving to: ‘hadoop_issues_inspected.xlsx’

hadoop_issues_inspe 100%[===================>] 134.43K  --.-KB/s    in 0.02s   

2025-04-29 14:40:58 (6.46 MB/s) - ‘hadoop_issues_inspected.xlsx’ saved [137652/137652]



In [40]:
df_hadoop_issues_inspected = pd.read_excel('hadoop_issues_inspected.xlsx')
df_hadoop_issues_inspected = transform_dataset(coluna_uteis, minhas_colunas, df_hadoop_issues_inspected)

<ipython-input-9-2b8ca721929e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_issues_inspected['label'] = df_x_issues_inspected['architectural_impact'].apply(lambda x: 1 if x == 'YES' else 0)
<ipython-input-9-2b8ca721929e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_issues_inspected['Textual_Type'] = 'AI_Yes'
<ipython-input-9-2b8ca721929e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [41]:
df_hadoop_issues_inspected.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,HADOOP-6252,Provide method to determine if a deprecated ke...,provide method determine deprecated key set co...,AI_No
1,HADOOP-6184,Provide a configuration dump in json format. ...,provide configuration dump json format . confi...,AI_Yes
2,HADOOP-6165,Add metadata to Serializations The Serializat...,add metadata serializations serialization fram...,AI_Yes
3,HADOOP-6161,Add get/setEnum to Configuration It would be ...,add get/setenum configuration would useful con...,AI_No
4,HADOOP-6103,Configuration clone constructor does not clone...,configuration clone constructor clone members ...,AI_No


## Cleaning diretories

In [42]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Create a StratifiedKFold

In [43]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_hadoop_issues_inspected['processed_text'], df_hadoop_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

skf.split(X, y)

<generator object _BaseKFold.split at 0x7e13e41a83a0>

## Performing tests in the model


In [44]:
list_my_metrics = perform_tests_in_model(X,y, label2id=label2id)

Fold 1: Train Size 118 | Test Size 30


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 1=> PRE: 0.6666666666666666; REC: 1.0; ACC: 0.6666666666666666; F1S: 0.8; AUC: 0.81
Fold 2: Train Size 118 | Test Size 30


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss


Fold 4=> PRE: 0.6551724137931034; REC: 1.0; ACC: 0.6551724137931034; F1S: 0.7916666666666666; AUC: 0.7631578947368423
Fold 5: Train Size 119 | Test Size 29


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 5=> PRE: 0.6551724137931034; REC: 1.0; ACC: 0.6551724137931034; F1S: 0.7916666666666666; AUC: 0.6421052631578947


## Hadoop issues results


In [45]:
df_my_metrics_hadoop_issues = pd.DataFrame(list_my_metrics)
df_my_metrics_hadoop_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.666667,1.000000,0.666667,0.800000,0.810000
1,2,0.666667,1.000000,0.666667,0.800000,0.525000
2,3,0.620690,0.947368,0.600000,0.750000,0.636364
3,4,0.655172,1.000000,0.655172,0.791667,0.763158
4,5,0.655172,1.000000,0.655172,0.791667,0.642105


# D) Testes de Inspeção de Issues do ActiveMQ

## Original Dataset

In [46]:
!wget https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/activemq_issues_inspected.xlsx

--2025-04-29 14:45:53--  https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/activemq_issues_inspected.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93147 (91K) [application/octet-stream]
Saving to: ‘activemq_issues_inspected.xlsx’

activemq_issues_ins 100%[===================>]  90.96K  --.-KB/s    in 0.02s   

2025-04-29 14:45:53 (5.74 MB/s) - ‘activemq_issues_inspected.xlsx’ saved [93147/93147]



In [47]:
df_activemq_issues_inspected = pd.read_excel('activemq_issues_inspected.xlsx')
df_activemq_issues_inspected = transform_dataset(coluna_uteis, minhas_colunas, df_activemq_issues_inspected)

<ipython-input-9-2b8ca721929e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_issues_inspected['label'] = df_x_issues_inspected['architectural_impact'].apply(lambda x: 1 if x == 'YES' else 0)


In [48]:
df_activemq_issues_inspected.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,AMQ-2149,Shared Filesystem Master Slave: missing messag...,shared filesystem master slave : missing messa...,AI_Yes
1,AMQ-2128,close() from MessageListener.onMessage() with ...,close ( ) messagelistener.onmessage ( ) auto d...,AI_No
2,AMQ-2106,Allow broker to evenly distribute message grou...,allow broker evenly distribute message groups ...,AI_Yes
3,AMQ-2103,Memory leak when marshaling ActiveMQTextMessag...,memory leak marshaling activemqtextmessage per...,AI_No
4,AMQ-2087,Redelivery after rollback does not seem to wor...,redelivery rollback seem work well test case r...,AI_No


## Cleaning diretories

In [49]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Create a StratifiedKFold

In [50]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_activemq_issues_inspected['processed_text'], df_activemq_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

skf.split(X, y)

<generator object _BaseKFold.split at 0x7e13e41a8820>

## Performing tests in model

In [51]:
list_my_metrics_activemq = perform_tests_in_model(X,y, label2id=label2id)

Fold 1: Train Size 105 | Test Size 27


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 1=> PRE: 0.6153846153846154; REC: 1.0; ACC: 0.6296296296296297; F1S: 0.7619047619047619; AUC: 0.44886363636363635
Fold 2: Train Size 105 | Test Size 27


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss


Fold 4=> PRE: 0.5769230769230769; REC: 1.0; ACC: 0.5769230769230769; F1S: 0.7317073170731707; AUC: 0.5636363636363637
Fold 5: Train Size 106 | Test Size 26


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 5=> PRE: 0.5769230769230769; REC: 1.0; ACC: 0.5769230769230769; F1S: 0.7317073170731707; AUC: 0.3878787878787878


## ActiveMQ issues results

In [52]:
df_my_metrics_activemq_issues = pd.DataFrame(list_my_metrics_activemq)
df_my_metrics_activemq_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.615385,1.0,0.629630,0.761905,0.448864
1,2,0.615385,1.0,0.629630,0.761905,0.613636
2,3,0.576923,1.0,0.576923,0.731707,0.539394
3,4,0.576923,1.0,0.576923,0.731707,0.563636
4,5,0.576923,1.0,0.576923,0.731707,0.387879


# E) Performing tests with Kafka, Hadoop and ActiveMQ

## Dataset

In [53]:
dataset = pd.concat([df_treino, df_teste], axis=0)
df_cassandra_issues_inspected = dataset[['issue_key', 'summary', 'description', 'comments_text', 'architectural_impact']]
df_cassandra_issues_inspected.rename(columns={'comments_text': 'comments'}, inplace=True)

<ipython-input-53-c70cb227e2b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected.rename(columns={'comments_text': 'comments'}, inplace=True)


In [54]:
df_cassandra_issues_inspected = transform_dataset(coluna_uteis, minhas_colunas, df_cassandra_issues_inspected)
df_cassandra_issues_inspected.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,CASSANDRA-3489,EncryptionOptions should be instantiated As th...,"encryptionoptions instantiated title says , ot...",AI_No
1,CASSANDRA-16780,Log when writing many tombstones to a partitio...,log writing many tombstones partition log writ...,AI_No
2,CASSANDRA-5426,Redesign repair messages Many people have been...,redesign repair messages many people reporting...,AI_Yes
3,CASSANDRA-5121,system.peers.tokens is empty after node restar...,system.peers.tokens empty node restart using 2...,AI_No
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...,AI_Yes


## Faz o merge dos datases

Cassandra, Kafka, Hadoop e ActiveMQ

In [55]:
df_all_issues_inspected = pd.concat([df_cassandra_issues_inspected,df_kafka_issues_inspected, df_hadoop_issues_inspected, df_activemq_issues_inspected], axis=0)
df_all_issues_inspected


,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,CASSANDRA-3489,EncryptionOptions should be instantiated As th...,"encryptionoptions instantiated title says , ot...",AI_No
1,CASSANDRA-16780,Log when writing many tombstones to a partitio...,log writing many tombstones partition log writ...,AI_No
2,CASSANDRA-5426,Redesign repair messages Many people have been...,redesign repair messages many people reporting...,AI_Yes
3,CASSANDRA-5121,system.peers.tokens is empty after node restar...,system.peers.tokens empty node restart using 2...,AI_No
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...,AI_Yes
...,...,...,...,...
127,AMQ-8413,Support different username and password for re...,support different username password remote bro...,AI_No
128,AMQ-9202,Reentrant locks should always be locked outsid...,reentrant locks always locked outside try bloc...,AI_No
129,AMQ-9157,Add a new advisory type for dispatched message...,add new advisory type dispatched messages use ...,AI_No
130,AMQ-9153,Fix Slow Consumer Advisory for Queue subscript...,fix slow consumer advisory queue subscriptions...,AI_No


## Cleaning diretories

In [56]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Create a StratifiedKFold

In [57]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_all_issues_inspected['processed_text'], df_all_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

skf.split(X, y)

<generator object _BaseKFold.split at 0x7e13e4538b80>

## Performing tests in the model

In [58]:
list_my_metrics_all_repositories = perform_tests_in_model(X,y, label2id=label2id)

Fold 1: Train Size 548 | Test Size 137


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 1=> PRE: 0.6521739130434783; REC: 0.8620689655172413; ACC: 0.6204379562043796; F1S: 0.7425742574257426; AUC: 0.6144827586206896
Fold 2: Train Size 548 | Test Size 137


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss


Fold 4=> PRE: 0.7096774193548387; REC: 0.7674418604651163; ACC: 0.656934306569343; F1S: 0.7374301675977654; AUC: 0.6906064751481987
Fold 5: Train Size 548 | Test Size 137


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Fold 5=> PRE: 0.6277372262773723; REC: 1.0; ACC: 0.6277372262773723; F1S: 0.7713004484304933; AUC: 0.6456908344733241


## All issues merged results

In [59]:
df_my_metrics_all_issues = pd.DataFrame(list_my_metrics_all_repositories)
df_my_metrics_all_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.652174,0.862069,0.620438,0.742574,0.614483
1,2,0.722222,0.597701,0.598540,0.654088,0.673793
2,3,0.637037,0.988506,0.635036,0.774775,0.605057
3,4,0.709677,0.767442,0.656934,0.737430,0.690606
4,5,0.627737,1.000000,0.627737,0.771300,0.645691


# Summary

## Cassandra Issues Results

In [60]:
df_my_metrics_cassandra_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.673913,1.000000,0.673913,0.805195,0.744086
1,2,0.688889,1.000000,0.688889,0.815789,0.776498
2,3,0.666667,1.000000,0.666667,0.800000,0.653333
3,4,0.666667,1.000000,0.666667,0.800000,0.744444
4,5,0.659091,0.966667,0.644444,0.783784,0.571111


## Kafka Issues Results

In [61]:
df_my_metrics_kafka_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.583333,1.000000,0.583333,0.736842,0.555556
1,2,0.583333,1.000000,0.583333,0.736842,0.736508
2,3,0.611111,1.000000,0.611111,0.758621,0.597403
3,4,0.611111,1.000000,0.611111,0.758621,0.477273
4,5,0.588235,0.952381,0.571429,0.727273,0.585034


## Hadoop Issues Results


In [62]:
df_my_metrics_hadoop_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.666667,1.000000,0.666667,0.800000,0.810000
1,2,0.666667,1.000000,0.666667,0.800000,0.525000
2,3,0.620690,0.947368,0.600000,0.750000,0.636364
3,4,0.655172,1.000000,0.655172,0.791667,0.763158
4,5,0.655172,1.000000,0.655172,0.791667,0.642105


## ActiveMQ Issues Results

In [63]:
df_my_metrics_activemq_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.615385,1.0,0.629630,0.761905,0.448864
1,2,0.615385,1.0,0.629630,0.761905,0.613636
2,3,0.576923,1.0,0.576923,0.731707,0.539394
3,4,0.576923,1.0,0.576923,0.731707,0.563636
4,5,0.576923,1.0,0.576923,0.731707,0.387879


## All Repositories Issues Resutls

In [64]:
df_my_metrics_all_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.652174,0.862069,0.620438,0.742574,0.614483
1,2,0.722222,0.597701,0.598540,0.654088,0.673793
2,3,0.637037,0.988506,0.635036,0.774775,0.605057
3,4,0.709677,0.767442,0.656934,0.737430,0.690606
4,5,0.627737,1.000000,0.627737,0.771300,0.645691
